### Recurrent Neural Networks

N-gram language models have major deficiency that they cannot capture long term dependencies in the text. Besides, as $n$ gets larger their storage requires too much space to use in daily-use computers.

Recurrent Neural Networks (RNNs) can consider all previous words in the sentence and can reflect longer dependencies. RNNs apply the same non-linear operations to words and store the result for future use as well. In other words, the the weights of the operations are the same for each word. In the below visualization, we see an example use case of RNN for completing a sentence. The model uses learnable $W_x$ matrix to operate on each word and pass the result of the operation to the next word using a different matrix $W_h$. The colors on the cells denote the result of the operations and how they accumulate until a prediction is made.

<img src='images/rnn.png' zoom='style: 20%' />

There are three types of RNNs: one-to-many, many-to-one, many-to-many. In the first type, RNN is fed with a single input and asked to generate a sequence. Example: image caption generation. In many-to-one type, the input is a sequence and the output is a single value, such as a class. Example: sentiment analysis. In the last category, both the input and the output are sequences. Example: machine translation. In this type, the input is process by an RNN named `encoder` and the its output is fed to another one called `decoder`.

The math of a vanilla RNN is quite simple. At each time step $t$, an input $x^{<t>}$ and the last hidden state $h^{<t-1>}$ are passed through a neural cell to create the next hidden state $h^{<t>}$ and an output $\hat{y}^{<t>$.

<img src='images/rnn_math.png' style='zoom: 30%' />

The equations can be written as follows:

$$h^{<t>}=g\left(W_{hh} h^{<t-1>} \oplus W_{hx} x^{<t>}+b_{h}\right)$$
$$\hat{y}^{<t>}=g\left(W_{yh} h^{<t> + b_y}\right)  $$

Here, we use three weight types for integrating previous hidden state to next one, the input, and prediction. The weights are shared between tokens to enable information sharing.

<img src='images/rnn_cell.png' style='zoom: 30%' />


As in any model, we need a loss function to train RNNs and we use cross-entropy averaged over time steps.

$$
J=-\frac{1}{T} \sum_{t=1}^{T} \sum_{j=1}^{K} y_{j}^{<t>} \log \hat{y}_{j}^{<t>}
$$

TensorFlow has a function named `scan` to apply the same operation to a list of elements and accumulate the results, as in RNNs. `scan` also accepts an initializer and returns the predictions and last hidden states.

```python
def scan(fn, elemn, initializer=None, ...):
    cur_value = initializer
    ys = []
    for x in elems:
        y, cur_value = fn(x, cur_value)
        ys.append(y)
        
    return ys, cur_value
```

### Gated Recurrent Units (GRUs)

Though RNNs are designed to capture long term dependencies, they suffer from vanishing gradient problem for longer sequences, leading model to forget the information related to initial tokens towards the end. GRUs are designed to tackle this problem, by explicitly learning what to forget at each step.

GRUs introduce a mechanism called gating to compute relevance of the hidden state and update only relevant parts of the hidden state. They use $\Gamme_{r}$ and $\Gamma_u$ gates to control information flow. $\Gamma_u$ is used to choose decide which part of the hidden state will be updated. It is computed as $\sigma \left(W_u [h^{<t_0}, x^{t_1}] + b_u \right)$. Here, $\sigma$ is the sigmoid function used to squeeze the output to $[0,1]$. $\Gamma_{r}$ is computed similarly, with a different weight matrix $W_r$.

These gates are used to update the hidden state. $\Gamma_r$ is used to compute an intermediary hidden state $h^{\prime<t>}= \tanh \left(W_{h}\left[\Gamma_{r} * h^{<t-1>}, x^{<t>}\right]+b_h\right)$. Then we compute $h^{<t>}$ using $h^{\prime<t>}$ and $\Gamma_u$. We pass the elements in $h^{\prime<t>}$ to $h^{<t>}$ if the correspondings element in $\Gamma_u$ is $0$. Otherwise, we use the element in the previous hidden state ($h^{<t-1>}$). It other words, we use the elements of $h^{<t-1>}$ in $h^{<t>}$ as well if the corresponding element in update gate $\Gamma_u$ is $1$. Thus, we explicitly learn if we need to update the previous hidden state. Last, we compute the prediction as in a Vanilla RNN.

We can list the equations as below, where $*$ denotes element-wise multiplication and $g(\cdot)$ is the softmax function.

$$
\begin{aligned}
\Gamma_{u} &=\sigma\left(W_{u}\left[h^{<t-1>}, x^{<t>}\right]+b_{u}\right) \\
\Gamma_{r} &=\sigma\left(W_{r}\left[h^{<t-1>}, x^{<t>}\right]+b_{r}\right) \\
h^{\prime<t>}=& \tanh \left(W_{h}\left[\Gamma_{r} * h^{<t-1>}, x^{<t>}\right]+b_h\right)\\
h^{<t>} &=\Gamma_{u} * h^{<t-1>}+\left(1-\Gamma_{u}\right) * h^{\prime<t>} \\
& \hat{y}^{<t>}=g\left(W_{y} h^{<t>} +b_{y}\right)
\end{aligned}
$$

<img src='images/gru_cell.png' style="zoom: 50%">

To improve these models, we can use a bidirectional structure that computes hidden states starting from the end, too. In other words, we process the sequence in the reverse order as well and then concatanate the hidden states during prediction.

Besides, we can stack RNNs as in feed-forward neural networks. This way, we can capture more subtle and complex relations in the input. 